In [ ]:
# NOTE: This uses a modified ethereum-etl here: https://github.com/MSilb7/optimism-etl

import sys
sys.path.append("../helper_functions")
import ethereumetl_utils as eetl
import web3py_utils as wpy
import pandas_utils as pu
import evm_utils as evm
sys.path.pop()

import pandas as pd
from dotenv import load_dotenv
import requests
import datetime
load_dotenv()
import os

In [ ]:
# Do we re-run data
run_data = 1
# Does l1_gas_used account for overhead? If not, set it to 0, if so, put it here (i.e. 600)
fixed_overhead = 0

configs = [
        # ['base_goerli',os.environ.get('BASE_GOERLI_NODE')]
        ['op_goerli_24h_load',os.environ.get('OP_GOERLI_TEAM_NODE')]
]

max_workers = 8
# Dune L1 Goerli Data https://dune.com/queries/2502137
# First Test 2023-05-23 05:00:00 - 2023-05-23 17:00:00 (12 hours)
# Forward Test 2023-05-23 17:01:00 - 2023-05-24 05:00:00 (12 hours)
start_timestamp = pu.convert_text_timestamp_to_int('2023-05-23 05:00:00')
end_timestamp = pu.convert_text_timestamp_to_int('2023-05-24 05:00:00')

print( pd.to_datetime(start_timestamp, unit='s') )
print( pd.to_datetime(end_timestamp, unit='s') )

In [ ]:
# Get Transactions
for i in configs:

    folder = i[0]
    uri = i[1]
#     block_arr = wpy.get_blockrange_by_timedelta(i[1],trailing_hours,'hours')
    st_bl = wpy.getBlockByTimestamp(i[1],start_timestamp)# block_arr[0]
    en_bl = wpy.getBlockByTimestamp(i[1],end_timestamp) # block_arr[1]
    print(st_bl)
    print(en_bl)
    if run_data == 1:
        eetl.get_ethereum_etl(st_bl,en_bl,folder,uri, max_workers)

In [ ]:
# Get Receipts
for i in configs:
    folder = i[0]
    uri = i[1]
    if run_data == 1:
        eetl.get_eth_etl_receipts(folder, uri)

### Process Results

In [ ]:
for i in configs:
        folder = i[0]
        uri = i[1]
        opg_df = pd.read_csv(os.getcwd() + '/downloads/' + folder + '/transactions.csv')
        rec_df = pd.read_csv(os.getcwd() + '/downloads/' + folder + '/receipts.csv')
        rec_df = rec_df.rename(columns={'transaction_hash':'hash'})
        rec_df = rec_df[['hash','l1_gas_used','l1_fee','l1_gas_price','l1_fee_scalar']]

        df = opg_df.merge(rec_df,on='hash',how='left')
        #REMOVE DSEPOSITS
        df = df[df['transaction_type'] != 126]
        df['all_data_gas'] = df['l1_gas_used'] - fixed_overhead
        df['input_calldata_gas'] = df['input'].apply(evm.count_calldata_gas)
        df['effective_l1_gas_used'] = df['l1_fee'] / df['l1_gas_price']
        df.to_csv(os.getcwd() + '/downloads/' + folder + '/merged_data.csv', index = False)

In [ ]:
group_cols = ['block_number','block_timestamp']

for i in configs:
        folder = i[0]
        uri = i[1]
        total_df = pd.read_csv(os.getcwd() + '/downloads/' + folder + '/merged_data.csv')

        blocks_df = total_df.groupby(group_cols).agg({
                'effective_l1_gas_used':'sum',
                'all_data_gas':'sum',
                'hash':'count',
                'l1_gas_price':'mean',
                'gas':'sum',
                'gas_price':'mean'
        }).reset_index()

        blocks_df = blocks_df.rename(columns={
                'effective_l1_gas_used':'total_l1_gas_used',
                'all_data_gas':'total_l1_data_gas_used',
                'hash':'num_txs',
                'l1_gas_price':'avg_l1_gas_price',
                'gas':'total_l2_gas_used',
                'gas_price':'avg_gas_price',
        })

        blocks_df['block_timestamp'] = pd.to_datetime(blocks_df['block_timestamp'], unit='s')
        display(blocks_df)

        blocks_df.to_csv(os.getcwd() + '/downloads/' + folder + '/by_block_summary.csv', index = False)